In [1]:
!pip install accelerate -U
!pip install transformers[torch]
!pip install transformers
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.1 MB/s eta 0:00:00


In [2]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer # baseline code용 tfidef vectorirzer(또는 tokenizer)
from sklearn import preprocessing
from sklearn.metrics import f1_score
from IPython.display import display, HTML

from transformers import AutoModel, AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer # 사용하고자 하는 모델, 토크나이저 적용시 필요
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, ClassLabel

from tqdm.auto import tqdm # process bar 표시용

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# 한글 깨짐 해결방법 : 실행 후 Restart runtime
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (9,646 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120500 files and dire

In [4]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic')

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device  # pt 코드에서 필요함

device(type='cpu')

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device  # pt 코드에서 필요함

device(type='cpu')

In [16]:
pretrained_model = 'monologg/kobigbird-bert-base'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

In [9]:
# 기사의 본문을 입력으로 받아서 해당 기사가 7개 topic중 어느 것에 속하는지를 분류
# loss(accuracy) 적용
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "IT과학", 1: "경제", 2: "사회", 3: "생활문화", 4: "세계", 5:"스포츠", 6: "정치" }
label2id = {"IT과학":0, "경제":1, "사회":2, "생활문화":3, "세계":4, "스포츠":5, "정치":6 }

In [10]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/nlpbook/best_model/topic_model')

In [12]:
model

BigBirdForSequenceClassification(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(32500, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdBlockSparseAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [17]:
def inference_fn(sentence) :
  inputs = tokenizer(sentence, truncation = True, return_token_type_ids = False, return_tensors = 'pt').to(device)
  model.eval()
  outputs = model(**inputs)
  predictions = outputs.logits.argsmax(-1)

  return {
      'sentence' : sentence,
      'prediction' : id2label[predictions.cpu().numpy([0])]
  }

In [18]:
sentence = '대통령이 국정연설을 한다.'
inference_fn(sentence)

Attention type 'block_sparse' is not possible if sequence_length: 9 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


AttributeError: ignored

In [ ]:
!mkdir  /root/.ngrok2 && echo 'authtoken: 2T5dj5MGKLVe9gdeTmRn96rqUGK_6bLTw98KhQnQENd1C9FCt' > /root/.ngrok2/ngrok.yml

In [ ]:
!pipi install flask_ngrok

In [ ]:
from flask import Flask, request, jsonify, render_template

def get_web_service_app(inference_fn, is_colab = True) :

  app = Flask(__name__, template_folder = '')
  if is_colab :
    from flask_ngrok import run_with_ngrok
    run_with_ngrok(app)

  else :
    from flask_cors import CORS
    CORS(app)

  @app.route('/')
  def index():
    return render_template('index.html')

  @app.route('/api', methods = ['post'])
  def api() :
    query_sentence = request.json['sentence']
    output_data = inference_fn(query_sentence)
    response = jsonify(output_data)
    return response
  return app

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
!rm *html

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

pretrained_model = 'monologg/kobigbird-bert-base'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

accuracy = load_metric("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "IT과학", 1: "경제", 2: "사회", 3: "생활문화", 4: "세계", 5:"스포츠", 6: "정치" }
label2id = {"IT과학":0, "경제":1, "사회":2, "생활문화":3, "세계":4, "스포츠":5, "정치":6 }

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("topic_model")
model.eval()

def inference_fn(sentence):
  inputs = tokenizer(sentence, truncation=True,  return_token_type_ids=False, return_tensors="pt").to(device)
  model.eval()
  outputs = model(**inputs)
  predictions = outputs.logits.argmax(-1) # argmax(-1)는 마지막 차원(일반적으로 클래스 차원)을 따라 가장 큰 값의 인덱스를 반환

  return {
    'sentence': sentence,
    'prediction': id2label[predictions.cpu().numpy()[0]],
  }
